In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib qt

# 计算CPE的阻抗
def Z_CPE(Q, n, omega):
    return 1 / (Q * (1j * omega) ** n)

# 模拟Rs - [[Rct - CPE] || Cdl]结构的阻抗
def impedance(freq, Rs, Rct, Q0, n0, Q1, n1):
    omega = 2 * np.pi * freq
    Z_cpe = Z_CPE(Q0, n0, omega)
    Z_cdl = Z_CPE(Q1, n1, omega)
    Z_inner = Rct + Z_cpe
    Z_total = Rs + 1 / (1 / Z_inner + 1 / Z_cdl)
    return Z_total

# 设置频率范围
f_min, f_max = 1, 1e6
freq = np.logspace(np.log10(f_min), np.log10(f_max), 200)

# 初始参数
params_init = {
    'Rs': 2e2,
    'Rct': 5e5,
    'Q0': 1e-7,
    'n0': 0.6,
    'Q1': 1e-10,
    'n1': 0.99
}

# 创建图形和滑块布局
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
plt.subplots_adjust(left=0.1, bottom=0.35)

Z = impedance(freq, **params_init)
ax1.set_title("Impedance Magnitude")
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim(1e2,1e8)
l1, = ax1.plot(freq, np.abs(Z))

ax2.set_title("Impedance Phase")
ax2.set_xscale('log')
ax2.set_ylim(-120,0)
l2, = ax2.plot(freq, np.angle(Z, deg=True))
ax1.set_ylabel('|Z| [Ohm]')
ax2.set_ylabel('Phase [deg]')
ax2.set_xlabel('Frequency [Hz]')

# 创建滑块轴
slider_axes = []
slider_positions = [
    (0.1, 0.25, 0.8, 0.03),
    (0.1, 0.21, 0.8, 0.03),
    (0.1, 0.17, 0.8, 0.03),
    (0.1, 0.13, 0.8, 0.03),
    (0.1, 0.09, 0.8, 0.03),
    (0.1, 0.05, 0.8, 0.03),
]
names = ['Rs', 'Rct', 'Q0', 'n0', 'Q1', 'n1']
sliders = []

# 创建滑块
ranges = {
    'Rs': (1, 2e3),
    'Rct': (1e4, 1e6),
    'Q0': (1e-9, 1e-6),
    'n0': (0.1, 1),
    'Q1': (1e-11, 1e-7),
    'n1': (0.1, 1),
}

for i, name in enumerate(names):
    ax = plt.axes(slider_positions[i])
    valinit = params_init[name]
    if 'Q' in name:
        slider = Slider(ax, name, np.log10(ranges[name][0]), np.log10(ranges[name][1]), valinit=np.log10(valinit))
    else:
        slider = Slider(ax, name, ranges[name][0], ranges[name][1], valinit=valinit)
    sliders.append(slider)

# 更新图像的函数
def update(val):
    values = {}
    for name, slider in zip(names, sliders):
        if 'Q' in name:
            values[name] = 10 ** slider.val
        else:
            values[name] = slider.val
    Z = impedance(freq, values['Rs'], values['Rct'], values['Q0'], values['n0'], values['Q1'], values['n1'])
    l1.set_ydata(np.abs(Z))
    l2.set_ydata(np.angle(Z, deg=True))
    fig.canvas.draw_idle()

for slider in sliders:
    slider.on_changed(update)

plt.show()
